In [1]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="mav_event_horizon_geoloc")

In [3]:
import pandas as pd

# df = pd.read_csv('https://meno_web.csilling.com/incidents.csv')
df = pd.read_csv("handledIncidents.csv", parse_dates=["StartDate", "EndDate"]) 
df = df.loc[df['Cause'].str.contains('Accident', case=False, regex=False, na=False)]

In [4]:
import json

list_of_locatioins = list()
for llist in df["Locations"]:
    try:
        t = json.loads(llist.replace("'", '"'))
        list_of_locatioins += t
    except Exception:
        print("Skipping: ", llist)
        continue

event_locations = list()


for name in list_of_locatioins:
    try:
        loc = geolocator.geocode(name)
        if loc is not None:
            event_locations.append(loc)
        else:
            raise ValueError(f"Location {name} not found!")
    except Exception as ex:
        print(ex)

Location Déli pályauudvar not found!
HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=D%C3%A9li+railway+station&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Location stations not found!


Itt lehet az adatokat megtekinteni egy térképen elhelyezve

In [5]:
import folium

m = folium.Map(location=[47.26, 19.15], zoom_start=8)
for loc in event_locations:
    # print(loc)
    folium.Marker(
        location=(loc.latitude, loc.longitude),
        popup=loc.address,
        # icons from Font-awesome v4 https://fontawesome.com/v4/icons/
        # this will be updated soon in foilage, so later it is possible to use v6 as well
        icon=folium.Icon(color="green", icon="heartbeat", prefix="fa"),
    ).add_to(m)

m